In [1]:
using Oceananigans
using Printf

In [2]:
N = 256
grid = RectilinearGrid(
    GPU(),
    size = (N, N, N),
    x = (0, N * π / 32),
    y = (0, N * π / 32),
    z = (0, N * π / 32),
    topology = (Periodic, Periodic, Periodic))

mrgrid = MultiRegionGrid(grid, partition = XPartition(2), devices = 2)

┌ Warning: MultiRegion functionalities are experimental: help the development by reporting bugs or non-implemented features!
└ @ Oceananigans.MultiRegion /home/ceoas/carlipp/.julia/packages/Oceananigans/qLQaY/src/MultiRegion/multi_region_grid.jl:106


MultiRegionGrid{Float64, Periodic, Periodic, Periodic} partitioned on GPU{CUDA.CUDAKernels.CUDABackend}(CUDA.CUDAKernels.CUDABackend(false, true)): 
├── grids: 128×256×256 RectilinearGrid{Float64, Oceananigans.Grids.FullyConnected, Periodic, Periodic} on CUDAGPU with 3×3×3 halo 
├── partitioning: Equal partitioning in X with (2 regions) 
├── connectivity: MultiRegionObject{Tuple{@NamedTuple{west::Oceananigans.MultiRegion.RegionalConnectivity{Oceananigans.MultiRegion.West, Oceananigans.MultiRegion.East}, east::Oceananigans.MultiRegion.RegionalConnectivity{Oceananigans.MultiRegion.East, Oceananigans.MultiRegion.West}, north::Nothing, south::Nothing}, @NamedTuple{west::Oceananigans.MultiRegion.RegionalConnectivity{Oceananigans.MultiRegion.West, Oceananigans.MultiRegion.East}, east::Oceananigans.MultiRegion.RegionalConnectivity{Oceananigans.MultiRegion.East, Oceananigans.MultiRegion.West}, north::Nothing, south::Nothing}}, Tuple{CUDA.CuDevice, CUDA.CuDevice}} 
└── devices: (CUDA.CuDevice(0

In [3]:
model = NonhydrostaticModel(; grid, advection = WENO())

e(x, y, z) = 2rand() - 1
set!(model, u = e, v = e, w = e)

simulation = Simulation(model; Δt = 0.01, stop_iteration = 100)

progress_message(sim) =
    @printf("Iteration: %04d, time: %s, Δt: %s, max(|w|) = %.1e ms⁻¹, wall time: %s\n",
        iteration(sim), prettytime(sim), prettytime(sim.Δt),
        maximum(abs, sim.model.velocities.w), prettytime(sim.run_wall_time))

add_callback!(simulation, progress_message, IterationInterval(20))

In [4]:
run!(simulation)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations /home/ceoas/carlipp/.julia/packages/Oceananigans/qLQaY/src/Simulations/run.jl:205


Iteration: 0000, time: 0 seconds, Δt: 10 ms, max(|w|) = 1.9e+00 ms⁻¹, wall time: 0 seconds


┌ Info:     ... simulation initialization complete (8.048 seconds)
└ @ Oceananigans.Simulations /home/ceoas/carlipp/.julia/packages/Oceananigans/qLQaY/src/Simulations/run.jl:248
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations /home/ceoas/carlipp/.julia/packages/Oceananigans/qLQaY/src/Simulations/run.jl:139
┌ Info:     ... initial time step complete (2.576 seconds).
└ @ Oceananigans.Simulations /home/ceoas/carlipp/.julia/packages/Oceananigans/qLQaY/src/Simulations/run.jl:168


Iteration: 0020, time: 200.000 ms, Δt: 10 ms, max(|w|) = 1.1e+00 ms⁻¹, wall time: 11.481 seconds
Iteration: 0040, time: 400.000 ms, Δt: 10 ms, max(|w|) = 8.7e-01 ms⁻¹, wall time: 12.572 seconds
Iteration: 0060, time: 600.000 ms, Δt: 10 ms, max(|w|) = 8.1e-01 ms⁻¹, wall time: 13.666 seconds
Iteration: 0080, time: 800.000 ms, Δt: 10 ms, max(|w|) = 6.8e-01 ms⁻¹, wall time: 14.762 seconds


┌ Info: Simulation is stopping after running for 15.858 seconds.
└ @ Oceananigans.Simulations /home/ceoas/carlipp/.julia/packages/Oceananigans/qLQaY/src/Simulations/simulation.jl:199
┌ Info: Model iteration 100 equals or exceeds stop iteration 100.
└ @ Oceananigans.Simulations /home/ceoas/carlipp/.julia/packages/Oceananigans/qLQaY/src/Simulations/simulation.jl:200


Iteration: 0100, time: 1000.000 ms, Δt: 10 ms, max(|w|) = 5.7e-01 ms⁻¹, wall time: 15.858 seconds


In [5]:
using CairoMakie

u, v, w = model.velocities
ζ = Field(∂x(v) - ∂y(u), indices = (:, :, grid.Nz))
compute!(ζ)
heatmap(ζ)

InitError: InitError: could not load library "/home/ceoas/carlipp/.julia/artifacts/f8704eb01ec7508ae1f125b7ff89647997ed80cb/lib/libgobject-2.0.so"
/home/ceoas/carlipp/.julia/artifacts/f8704eb01ec7508ae1f125b7ff89647997ed80cb/lib/libgobject-2.0.so: undefined symbol: g_dir_unref
during initialization of module Glib_jll